In [1]:
import json
from elasticsearch5 import Elasticsearch
from elasticsearch5.helpers import reindex

In [2]:
es = Elasticsearch(hosts="localhost:9200")

In [3]:
### Deleting the final index added for every time the program is run and index is created

es.indices.delete(index="shakespearefile1", ignore=404)
es.indices.delete(index="shakespearefile2", ignore=404)

{'acknowledged': True}

In [4]:
shakespeare_index = "shakespearefile1"

In [5]:
custom_analyzer_system1 = {
    "analysis": {
        "analyzer": {
            "my_custom_analyzer": {
                "type": "custom",
                "tokenizer": "standard",
                "filter": ["lowercase", "porter_stem"],
            }
        }
    }
}

document_mappings = {
    "properties": {
        "line_id": {"type": "long"},
        "play_name": {"type": "text"},
        "line_number": {"type": "text"},
        "speaker": {"type": "text"},
        "speech_number": {"type": "text"},
        "speaker": {"type": "text"},
        "text_entry": {
            "type": "text",
            "analyzer": "my_custom_analyzer"
        },
    }
}

elastic_search_config = {
    "settings": custom_analyzer_system1,
    "mappings": {
        "line": document_mappings,
        "scene": document_mappings,
        "act": document_mappings,
    }
}

index1 = es.indices.create(index=shakespeare_index, body=elastic_search_config, ignore=400)
index1

{'acknowledged': True,
 'shards_acknowledged': True,
 'index': 'shakespearefile1'}

In [6]:
shakespeare_index_2 = "shakespearefile2"

In [7]:
custom_analyzer_system2 = {
    "analysis": {
        "analyzer": {
            "my_custom_analyzer": {
                "type": "custom",
                "tokenizer": "standard",
                "filter": ["lowercase"],
            }
        }
    }
}

document_mappings_system2 = {
    "properties": {
        "line_id": {"type": "long"},
        "play_name": {"type": "text"},
        "line_number": {"type": "text"},
        "speaker": {"type": "text"},
        "speech_number": {"type": "text"},
        "speaker": {"type": "text"},
        "text_entry": {
            "type": "text",
            "analyzer": "my_custom_analyzer"
        },
    }
}

elastic_search_config_2 = {
    "settings": custom_analyzer_system2,
    "mappings": {
        "line": document_mappings_system2,
        "scene": document_mappings_system2,
        "act": document_mappings_system2,
    }
}

index2 = es.indices.create(index=shakespeare_index_2 , body=elastic_search_config_2, ignore=400)
index2



{'acknowledged': True,
 'shards_acknowledged': True,
 'index': 'shakespearefile2'}

In [8]:
shakespeare_destination_index = 'shakespeareReindex'

In [9]:
reindex_body = {
    "source": {
        "index": [index1, index2]
    },
    "dest": {
        "index": shakespeare_destination_index
    }
}

es.reindex(body=reindex_body, refresh=True, ignore=400)

{'error': {'root_cause': [{'type': 'parsing_exception',
    'reason': '[reindex] failed to parse field [source]',
    'line': 1,
    'col': 186}],
  'type': 'parsing_exception',
  'reason': '[reindex] failed to parse field [source]',
  'line': 1,
  'col': 186,
  'caused_by': {'type': 'array_store_exception', 'reason': None}},
 'status': 400}